In [1]:
import os
import websockets
import json
import pandas as pd
import asyncio
import ta
import numpy as np
from binance.client import Client
import time

In [2]:
# Initialise env variables
API_KEY = os.environ.get('BINANCE_API_KEY')
API_SECRET = os.environ.get('BINANCE_API_SECRET')

# Initialise Binance Client
client = Client(API_KEY, API_SECRET)

# Init buy / sell vars
COIN_NAME = 'SHIBUSDT'
BUY_AMOUNT_USD = float(10)

# Init stop loss / take profit
STOP_PROFIT = 1.005
STOP_LOSS = 0.995

COIN_PRICE = float(client.get_symbol_ticker(symbol=COIN_NAME)['price'])
print(f'Price of {COIN_NAME}: {COIN_PRICE} {COIN_NAME[3:-1]}')
BUY_AMOUNT_COIN = round(BUY_AMOUNT_USD / COIN_PRICE, 8)
print(f'Amount of {COIN_NAME} to buy: {BUY_AMOUNT_COIN} {COIN_NAME[0:-4]}')

Price of SHIBUSDT: 4.55e-05 BUSD
Amount of SHIBUSDT to buy: 219780.21978022 SHIB


In [3]:
def getMinuteData(symbol = COIN_NAME, interval = '1m', minutes = '60'):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, minutes + ' min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit = 'ms')
    frame = frame.astype(float)
    return frame

In [4]:
df = getMinuteData()

In [5]:
def applyTechnicals(df):
    df['%K'] = ta.momentum.stoch(df.High, df.Low, df.Close, window = 14, smooth_window = 3)
    df['%D'] = df['%K'].rolling(3).mean()
    df['RSI'] = ta.momentum.rsi(df.Close, window = 14)
    df['MACD'] = ta.trend.macd_diff(df.Close)
    df.dropna(inplace = True)

In [6]:
applyTechnicals(df)

In [7]:
class Signals:
    
    def __init__(self, df, lags):
        self.df = df
        self.lags = lags
        
    def getTrigger(self):
        dfx = pd.DataFrame()
        for i in range(self.lags + 1):
            mask = (self.df['%K'].shift(i) < 20) & (self.df['%D'].shift(i) < 20)
            dfx = dfx.append(mask, ignore_index = True)
        return dfx.sum(axis = 0)
    
    def decide(self):
        self.df['trigger'] = np.where(self.getTrigger(), 1, 0)
        self.df['Buy'] = np.where((self.df.trigger) & (self.df['%K'].between(20,80)) & 
                                  (self.df['%D'].between(20,80)) & (self.df.RSI > 50) & 
                                  (self.df.MACD > 0), 1, 0)

In [8]:
inst = Signals(df, 10)

In [9]:
inst.decide()

In [51]:
df[df.Buy == 1]

,Open,High,Low,Close,Volume,%K,%D,RSI,MACD,trigger,Buy
Time,,,,,,,,,,,


In [10]:
def strategy(coin, qty, open_position = False):
    # Testing
    profit = 0.0
    buy_price = 0.0
    
    df = getMinuteData(coin, '1m', '100')
    applyTechnicals(df)
    inst = Signals(df, 5)
    inst.decide()
    # print(f'Current Close: ' + str(df.Close.iloc[-1]))

    # Look to buy
    if df.Buy.iloc[-1]:
        #order = client.create_order(symbol = coin, side = 'BUY', type = 'MARKET', quantity = qty)
        #print(order)
        #buyPrice = float(order['fills'][0]['price'])
        open_position = True
        
        # Testing
        buy_price = df.Close.iloc[-1]
        print(f'Pretending to BUY @ {buy_price}')
    
    while open_position:
        time.sleep(0.5)
        df = getMinuteData(coin, '1m', '2')
        #print('-------')
        # print(f'Current Close: ' + str(df.Close.iloc[-1]) + str(df.Close[-1]))
        # print(f'Stop_Profit: ' + str(buy_price * STOP_PROFIT))
        # print(f'Stop_Loss: ' + str(buy_price * STOP_LOSS))
        # print('-------')
        
        # Check to exit trade 
        if df.Close[-1] <= buy_price * STOP_LOSS or df.Close[-1] >= buy_price * STOP_PROFIT:
            #order = client.create_order(symbol = coin, side = 'SELL', type = 'MARKET', quantity = qty)
            #print(order)
            
            # Testing
            profit = df.Close[-1] - buy_price
            print(f'Pretending to SELL @ {df.Close[-1]}')
            
            break
    return profit

In [ ]:
total = 0.0
tradeNum = 1
print('Trading...')
while True:
    tradeProfit = float(strategy('ADAUSDT', 10))
    total = total + tradeProfit
    
    if tradeProfit != 0:
        print(f'Profit for trade #{tradeNum} = {tradeProfit}')
        print(f'Total Profit: {total}')
        print('--------------------')
        tradeNum += 1
    
    time.sleep(0.5)